The goal of this Notebook is to create pyspark queris on cars made in germany in order to test the performance of pyspark and also see its capabilites and performance in EDA. The base goal is tp writes 25 queries answering question about our 46405 rows × 9 columns data

In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f651e1b516f1e4adc4c098da885eeac68ec624333de45d726edd1e35ef225d4b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Import libraries

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions  as f
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row

In [ ]:
Data =  pd.read_csv("cars_Germany.csv")

In [ ]:
Data

,mileage,make,model,fuel,gear,offerType,price,hp,year
0,235000,BMW,316,Diesel,Manual,Used,6800,116.0,2011
1,92800,Volkswagen,Golf,Gasoline,Manual,Used,6877,122.0,2011
2,149300,SEAT,Exeo,Gasoline,Manual,Used,6900,160.0,2011
3,96200,Renault,Megane,Gasoline,Manual,Used,6950,110.0,2011
4,156000,Peugeot,308,Gasoline,Manual,Used,6950,156.0,2011
...,...,...,...,...,...,...,...,...,...
46400,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46401,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46402,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46403,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021


In [ ]:
sc = SparkContext.getOrCreate()
sqlsess = SparkSession(sc)


Data in PySpark is represented as RDDs, which stands for Resilient Distributed Datasets. These are immutable, partitioned collections of objects spread across a cluster of machines that can be operated on in parallel. RDDs have advanced the field of big data computations because they allow for in-memory caching, which is a more efficient solution than the read and write operations to the disk used in Hadoop's HDFS system, where disk I/O was considered a significant overhead.

In [ ]:

data = sqlsess.read.option("header","true").csv("cars_Germany.csv")

data

DataFrame[mileage: string, make: string, model: string, fuel: string, gear: string, offerType: string, price: string, hp: string, year: string]

Now our data is stored as Pyspark Dataframe which are key contribution to pyspark

If we want our data to be as rdd, we can do the following

In [ ]:
data_rdd = data.rdd
data_rdd

MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0

In here, i will show the data as dataframe in pyspark, where i will later use the dataframe API in order to run queries and perform the transformations and axctions of my ETL journery on the data.

In [ ]:
data.show(10)

+-------+----------+-------+-----------------+---------+---------+-----+---+----+
|mileage|      make|  model|             fuel|     gear|offerType|price| hp|year|
+-------+----------+-------+-----------------+---------+---------+-----+---+----+
| 235000|       BMW|    316|           Diesel|   Manual|     Used| 6800|116|2011|
|  92800|Volkswagen|   Golf|         Gasoline|   Manual|     Used| 6877|122|2011|
| 149300|      SEAT|   Exeo|         Gasoline|   Manual|     Used| 6900|160|2011|
|  96200|   Renault| Megane|         Gasoline|   Manual|     Used| 6950|110|2011|
| 156000|   Peugeot|    308|         Gasoline|   Manual|     Used| 6950|156|2011|
| 147000|    Toyota|  Auris|Electric/Gasoline|Automatic|     Used| 6950| 99|2011|
|  91894|   Renault| Scenic|           Diesel|   Manual|     Used| 6970|131|2011|
| 127500|      Opel| Zafira|         Gasoline|   Manual|     Used| 6972|116|2011|
| 115000|     Mazda|      3|         Gasoline|Automatic|     Used| 6980|150|2011|
|    104|      F

select query
Note that we are running the action .show() in order to get the execution immediatly

we will use both .where() and .filter() clauses.

In [ ]:
data.select("mileage").show()

+-------+
|mileage|
+-------+
| 235000|
|  92800|
| 149300|
|  96200|
| 156000|
| 147000|
|  91894|
| 127500|
| 115000|
|    104|
|  59580|
|  84000|
| 427000|
| 109000|
| 194020|
|  72000|
| 190000|
| 190000|
|  85700|
|  82000|
+-------+
only showing top 20 rows



Conditioing

show make, model and price for cars that cost more than 60 000 euros

In [ ]:
data.select("make","model","price")\
    .where(f.col("price") > 60000)\
    .show()

+-------------+-------------+-----+
|         make|        model|price|
+-------------+-------------+-----+
|         Audi|        TT RS|64885|
|       Jaguar|       F-Type|64900|
|         Audi|        TT RS|65890|
|      Porsche|        Macan|65900|
|   Volkswagen|      Touareg|66740|
|         Audi|          SQ5|66790|
|         Audi|           A7|66890|
|         Audi|          SQ5|67840|
|         Audi|           A7|69890|
|         Audi|        RS Q3|69890|
|         Audi|           S5|69980|
|   Volkswagen|T6 California|70980|
|         Audi|           Q7|81890|
|         Audi|           Q8|81890|
|         Audi|           Q8|81890|
|         Audi|           Q8|83890|
|   Volkswagen|      Touareg|61950|
|        Volvo|         XC60|64800|
|Mercedes-Benz|      CLS 350|66890|
|   Volkswagen|T6 California|74990|
+-------------+-------------+-----+
only showing top 20 rows



In [ ]:
data.select("make","model","price")\
    .filter(f.col("price") >60000)\
    .show()

+-------------+-------------+-----+
|         make|        model|price|
+-------------+-------------+-----+
|         Audi|        TT RS|64885|
|       Jaguar|       F-Type|64900|
|         Audi|        TT RS|65890|
|      Porsche|        Macan|65900|
|   Volkswagen|      Touareg|66740|
|         Audi|          SQ5|66790|
|         Audi|           A7|66890|
|         Audi|          SQ5|67840|
|         Audi|           A7|69890|
|         Audi|        RS Q3|69890|
|         Audi|           S5|69980|
|   Volkswagen|T6 California|70980|
|         Audi|           Q7|81890|
|         Audi|           Q8|81890|
|         Audi|           Q8|81890|
|         Audi|           Q8|83890|
|   Volkswagen|      Touareg|61950|
|        Volvo|         XC60|64800|
|Mercedes-Benz|      CLS 350|66890|
|   Volkswagen|T6 California|74990|
+-------------+-------------+-----+
only showing top 20 rows



Average price for each offertype made on BMW cars.

I will run this using SQL and API dataframe

In [ ]:
# create a view for make, price and offer type

first_round = data.select("make",  "offertype", "price")\
              .filter(f.col("make") == "BMW")\
              .groupBy("offertype")\
              .agg(f.round(f.avg("price"),2).alias("Average price"))\



In [ ]:
first_round.collect()

[Row(offertype="Employee's car", Average price=48145.38),
 Row(offertype='Used', Average price=22357.67),
 Row(offertype='Pre-registered', Average price=41450.0),
 Row(offertype='Demonstration', Average price=65677.58)]

In [ ]:
first_round.show()

+--------------+-------------+
|     offertype|Average price|
+--------------+-------------+
|Employee's car|     48145.38|
|          Used|     22357.67|
|Pre-registered|      41450.0|
| Demonstration|     65677.58|
+--------------+-------------+



Now make it as Sql query.

In [ ]:
data.createOrReplaceTempView("Cars")

In [ ]:
res = sqlsess.sql(
   """ select offertype, round(avg(price),2) as Average_price
       from Cars
       where make = "BMW"
       group by offertype

   """
)

res.show()

+--------------+-------------+
|     offertype|Average_price|
+--------------+-------------+
|Employee's car|     48145.38|
|          Used|     22357.67|
|Pre-registered|      41450.0|
| Demonstration|     65677.58|
+--------------+-------------+



Now i want to do aggregation and conditioning and both of them.

In [ ]:
data.printSchema()

root
 |-- mileage: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- year: string (nullable = true)



After seing the data i have noteced i problem which is price and year and mileage are string instead of int
i will use withcolumns and cast function in order to solve this problem.

In [ ]:
new_data = data.withColumn("price", f.col("price").cast("int"))\
               .withColumn("mileage", f.col("mileage").cast("int"))\
               .withColumn("year", f.col("year").cast("int"))


In [ ]:
new_data.printSchema()

root
 |-- mileage: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- hp: string (nullable = true)
 |-- year: integer (nullable = true)



In [ ]:
new_data.createOrReplaceTempView("CarsT")

In [ ]:
# i want to get the car that their milage is less than 1000 and they are Automatic then see their max, min and average price

cars_like_new = new_data.where((f.col("mileage") <= 1000) & (f.col("gear") == "Automatic"))

In [ ]:
cars_like_new.show(100)

+-------+-------------+---------------+-----------------+---------+--------------+------+---+----+
|mileage|         make|          model|             fuel|     gear|     offerType| price| hp|year|
+-------+-------------+---------------+-----------------+---------+--------------+------+---+----+
|    999|         Audi|             A7|Electric/Gasoline|Automatic|          Used| 66890|367|2020|
|      5|          Kia|         Optima|         Gasoline|Automatic| Demonstration| 28590|179|2019|
|     10|      Hyundai|          Ioniq|Electric/Gasoline|Automatic| Demonstration| 21250|105|2020|
|     29|         Ford|           Puma|         Gasoline|Automatic|          Used| 21490|125|2020|
|     10|      Hyundai|            i30|         Gasoline|Automatic| Demonstration| 21540|140|2020|
|      5|      Hyundai|         Tucson|         Gasoline|Automatic|Pre-registered| 36870|179|2021|
|     10|      Hyundai|            i30|         Gasoline|Automatic|Pre-registered| 36990|280|2021|
|     10| 

In [ ]:
# lets see the cheapest automatic cars
cars_like_new.where(f.col("price") < 10000).show()

+-------+-------+-----+--------+---------+--------------+-----+----+----+
|mileage|   make|model|    fuel|     gear|     offerType|price|  hp|year|
+-------+-------+-----+--------+---------+--------------+-----+----+----+
|    150| Others| NULL|Electric|Automatic|Employee's car| 4490|NULL|2020|
|   1000| Others| NULL|Electric|Automatic|          Used| 3580|   5|2012|
|    990|Renault|Twizy|Electric|Automatic|          Used| 4690|  11|2012|
|      0| Others| NULL|Electric|Automatic|          Used| 5999|   3|2019|
|     40| Others| NULL|Electric|Automatic|          Used| 9870|  52|2019|
|    300| Others| NULL|Electric|Automatic|          Used| 7000|   5|2019|
|    100|Renault|  ZOE|Electric|Automatic|Pre-registered| 9880|  72|2020|
+-------+-------+-----+--------+---------+--------------+-----+----+----+



In [ ]:
cars_like_new_pricess = cars_like_new.agg(
    f.min(f.col("price")).alias("min_price"),
    f.max(f.col("price")).alias("max_price"),
    f.round(f.avg(f.col("price")),2).alias("avg_price")
)

# Show the aggregated results
cars_like_new_pricess.show()

+---------+---------+---------+
|min_price|max_price|avg_price|
+---------+---------+---------+
|     3580|  1199900| 37131.95|
+---------+---------+---------+



In [ ]:
cars_like_new_prices = data.select("mileage", "gear", "price") \
                        .where((f.col("mileage") < 1000) & (f.col("gear") == "Automatic"))

In [ ]:
cars_like_new_prices.show(100)

+---------+---------+------------------+
|min_price|max_price|         avg_price|
+---------+---------+------------------+
|   102800|     9880|37133.627057981386|
+---------+---------+------------------+



In [ ]:
res = sqlsess.sql(
    """
    select min(price) as min , max(price) as max , round(avg(price),2) as avg
    from  CarsT
    where mileage < 1000 and gear = "Automatic"
    """
)

In [ ]:
res.show()

+----+-------+--------+
| min|    max|     avg|
+----+-------+--------+
|4490|1199900|37133.63|
+----+-------+--------+



get average price per model:


In [ ]:
price_per_model = new_data.groupBy(f.col("make")) \
                          .agg(f.round(f.avg(f.col("price")),2).alias("avg_price"))

In [ ]:
price_per_model.show()

+--------------+---------+
|          make|avg_price|
+--------------+---------+
|        Zhidou|  12500.0|
|    Volkswagen| 16065.93|
|      Infiniti|  16329.6|
|       Peugeot|  9942.04|
|        Alpina|  66046.3|
|         Lexus| 42299.17|
|      Polestar|  64150.0|
|        Jaguar| 42385.33|
|      Maserati| 72480.58|
|   Rolls-Royce|196266.67|
|         Aixam| 16016.67|
|Caravans-Wohnm| 19146.67|
|          Jeep| 28454.14|
|        Lancia|   6295.5|
|    Mitsubishi| 10181.47|
|           9ff|   7000.0|
|           Kia|  13403.2|
|     Chevrolet|  6699.75|
|       Piaggio|   7077.8|
|    Trucks-Lkw|   8200.0|
+--------------+---------+
only showing top 20 rows



In [ ]:
numbers_of_companies = new_data.agg(f.countDistinct("make").alias("number_companies")).show()

+----------------+
|number_companies|
+----------------+
|              77|
+----------------+



do them in SQL

In [ ]:
res3 = sqlsess.sql(
    """
    select make, round(avg(price),2) as avg_price
    from carsT
    group by make
    """
)

In [ ]:
res3.show()

+--------------+---------+
|          make|avg_price|
+--------------+---------+
|        Zhidou|  12500.0|
|    Volkswagen| 16065.93|
|      Infiniti|  16329.6|
|       Peugeot|  9942.04|
|        Alpina|  66046.3|
|         Lexus| 42299.17|
|      Polestar|  64150.0|
|        Jaguar| 42385.33|
|      Maserati| 72480.58|
|   Rolls-Royce|196266.67|
|         Aixam| 16016.67|
|Caravans-Wohnm| 19146.67|
|          Jeep| 28454.14|
|        Lancia|   6295.5|
|    Mitsubishi| 10181.47|
|           9ff|   7000.0|
|           Kia|  13403.2|
|     Chevrolet|  6699.75|
|       Piaggio|   7077.8|
|    Trucks-Lkw|   8200.0|
+--------------+---------+
only showing top 20 rows



In [ ]:
res4 = sqlsess.sql(
    """
    select count(distinct make) as number_companies
    from carsT
   """
)

In [ ]:
res4.show()

+----------------+
|number_companies|
+----------------+
|              77|
+----------------+



I saw that the fuel table has Electric/Gasoline options i want to split them and create a table for each of them or wherever / occurs

In [ ]:
special_char_infuel = new_data.select(f.col("fuel")) \
                              .select(f.col("fuel"), f.split(f.col("fuel") ,"/").alias("splitted"))\
                              .select(f.col("fuel"), f.col("splitted")[0].alias("first_opt"), f.col("splitted")[1].alias("second_opt") )\
                              .where((f.col("fuel").contains('/')) & (~f.col("fuel").contains('-')))


In [ ]:
special_char_infuel.show()

+-----------------+---------+----------+
|             fuel|first_opt|second_opt|
+-----------------+---------+----------+
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|  Electric/Diesel| Electric|    Diesel|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
+-----------------+---------+----------+
only showing top

In [ ]:
#count how many option there are in the first and second options
special_char_infuel.select(f.countDistinct(f.col("first_opt")).alias("count_first_opt"),
                           f.countDistinct(f.col("second_opt")).alias("count_second_opt")
                           ).show()

+---------------+----------------+
|count_first_opt|count_second_opt|
+---------------+----------------+
|              1|               2|
+---------------+----------------+



Do the same in SQL:

In [ ]:
res5 = sqlsess.sql(
    """

  select fuel, split(fuel, "/")[0] as first_opt, split(fuel, "/")[1] as second_opt
      from carsT
    where fuel like "%/%" and fuel not like "%-%"
    """
)

In [ ]:
res5.show()

+-----------------+---------+----------+
|             fuel|first_opt|second_opt|
+-----------------+---------+----------+
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|  Electric/Diesel| Electric|    Diesel|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
|Electric/Gasoline| Electric|  Gasoline|
+-----------------+---------+----------+
only showing top

In [ ]:
res6 = sqlsess.sql(
    """
    With split_values as(

      select fuel, split(fuel, "/")[0] as first_opt, split(fuel, "/")[1] as second_opt
      from carsT
    where fuel like "%/%" and fuel not like "%-%"

    )

    select count(Distinct first_opt) as count_first_opt, count(Distinct second_opt) as count_second_opt
    from split_values

    """
)

In [ ]:
res6.show()

+---------------+----------------+
|count_first_opt|count_second_opt|
+---------------+----------------+
|              1|               2|
+---------------+----------------+



I still need to do concatinating and goal checked for today:

In [ ]:
new_data.show()

+-------+-------------+------------+-----------------+---------+---------+-----+---+----+
|mileage|         make|       model|             fuel|     gear|offerType|price| hp|year|
+-------+-------------+------------+-----------------+---------+---------+-----+---+----+
| 235000|          BMW|         316|           Diesel|   Manual|     Used| 6800|116|2011|
|  92800|   Volkswagen|        Golf|         Gasoline|   Manual|     Used| 6877|122|2011|
| 149300|         SEAT|        Exeo|         Gasoline|   Manual|     Used| 6900|160|2011|
|  96200|      Renault|      Megane|         Gasoline|   Manual|     Used| 6950|110|2011|
| 156000|      Peugeot|         308|         Gasoline|   Manual|     Used| 6950|156|2011|
| 147000|       Toyota|       Auris|Electric/Gasoline|Automatic|     Used| 6950| 99|2011|
|  91894|      Renault|      Scenic|           Diesel|   Manual|     Used| 6970|131|2011|
| 127500|         Opel|      Zafira|         Gasoline|   Manual|     Used| 6972|116|2011|
| 115000| 

In [ ]:
new_data.select( f.concat_ws(" ", (f.col("make")), f.col("model") ) ).show()

+-------------------------+
|concat_ws( , make, model)|
+-------------------------+
|                  BMW 316|
|          Volkswagen Golf|
|                SEAT Exeo|
|           Renault Megane|
|              Peugeot 308|
|             Toyota Auris|
|           Renault Scenic|
|              Opel Zafira|
|                  Mazda 3|
|             Ford Transit|
|              Opel Meriva|
|          Volkswagen Golf|
|      Mercedes-Benz E 250|
|        Chevrolet Orlando|
|                  Audi A4|
|          Volkswagen Polo|
|           Renault Espace|
|     Renault Grand Espace|
|              Fiat Sedici|
|               Opel Corsa|
+-------------------------+
only showing top 20 rows



With SQL

In [ ]:
Res7 = sqlsess.sql(

      """
      select concat(make," ", model) from carst

      """

)
Res7.show()

+----------------------+
|concat(make,  , model)|
+----------------------+
|               BMW 316|
|       Volkswagen Golf|
|             SEAT Exeo|
|        Renault Megane|
|           Peugeot 308|
|          Toyota Auris|
|        Renault Scenic|
|           Opel Zafira|
|               Mazda 3|
|          Ford Transit|
|           Opel Meriva|
|       Volkswagen Golf|
|   Mercedes-Benz E 250|
|     Chevrolet Orlando|
|               Audi A4|
|       Volkswagen Polo|
|        Renault Espace|
|  Renault Grand Espace|
|           Fiat Sedici|
|            Opel Corsa|
+----------------------+
only showing top 20 rows



List make



companies that only have Manual cars:


In [ ]:
data.groupBy("make").agg(g.collect_set("gear").alias("types"))\
                    .filter((f.size("types") == 1) & f.array_contains(f.col("types"), "Automatic") )

In [ ]:
data1 = new_data.groupBy("make").agg(f.collect_set("gear").alias("collected_gear"))\
                                .select("make")\
                                .where((f.size("collected_gear")==1) & (f.array_contains(f.col("collected_gear"),"Automatic")))
data1.show()

+-----------+
|       make|
+-----------+
|     Zhidou|
|     Alpina|
|   Polestar|
|   Maserati|
|Rolls-Royce|
|      Aixam|
|        RAM|
|Lamborghini|
|     Ligier|
|    Estrima|
|    McLaren|
|      Tesla|
|   Chrysler|
|    Ferrari|
|    Bentley|
|    Maybach|
|     FISKER|
|       DFSK|
+-----------+



In [ ]:
res8 = sqlsess.sql(
    """
    SELECT make
    FROM (
        SELECT
            make,
            COUNT(DISTINCT CASE WHEN gear = 'Automatic' THEN gear ELSE NULL END) AS automatic_count,
            COUNT(DISTINCT gear) AS total_gear_count
        FROM carst
        GROUP BY make
    ) AS subquery
    WHERE automatic_count = 1 AND total_gear_count = 1;

    """
)

In [ ]:
res8.show()

+-----------+
|       make|
+-----------+
|     Zhidou|
|     Alpina|
|   Polestar|
|   Maserati|
|Rolls-Royce|
|      Aixam|
|        RAM|
|Lamborghini|
|     Ligier|
|    Estrima|
|    McLaren|
|      Tesla|
|   Chrysler|
|    Ferrari|
|    Bentley|
|    Maybach|
|     FISKER|
|       DFSK|
+-----------+



For each fuel type, I want to find the make and model that has the highest average horsepower to price ratio, indicating which cars offer more power per unit of price among those fueled similarly.

Price Ratio= Price / Horsepower (hp)
**​**

In [ ]:
price_ratio = new_data.groupBy("make", "model")\
                      .agg(f.round((f.avg("price")/f.avg("hp")),2).alias("price_ratio"))\
                      .sort(f.col("price_ratio").desc())\
                      .filter((f.col("make")!= "NULL") & (f.col("model") != "NULL") )


Worst 10

In [ ]:
price_ratio.head(10)

[Row(make='Estrima', model='Birò', price_ratio=2618.0),
 Row(make='Aixam', model='City', price_ratio=1990.0),
 Row(make='Ligier', model='JS 50', price_ratio=1914.0),
 Row(make='Ferrari', model='F12', price_ratio=1548.26),
 Row(make='Microcar', model='M.Go', price_ratio=1410.73),
 Row(make='Mercedes-Benz', model='S 650', price_ratio=1138.22),
 Row(make='Maybach', model='Pullman', price_ratio=1049.03),
 Row(make='Microcar', model='M8', price_ratio=1024.92),
 Row(make='Microcar', model='Due', price_ratio=899.33),
 Row(make='Morgan', model='3-Wheeler', price_ratio=822.06)]

Best 10

In [ ]:
price_ratio.tail(10)

[Row(make='Nissan', model='Pathfinder', price_ratio=36.84),
 Row(make='Renault', model='Latitude', price_ratio=35.23),
 Row(make='Chrysler', model='Grand Voyager', price_ratio=34.84),
 Row(make='Dodge', model='Journey', price_ratio=32.14),
 Row(make='Lancia', model='Thema', price_ratio=27.94),
 Row(make='Kia', model='Carnival', price_ratio=26.33),
 Row(make='Others', model='Landwind', price_ratio=19.07),
 Row(make='Brilliance', model='BS4', price_ratio=14.85),
 Row(make='Lada', model='Priora', price_ratio=12.14),
 Row(make='Caravans-Wohnm', model='Adria', price_ratio=None)]

Using SQL

In [ ]:
Res9 = sqlsess.sql(
    """
    Select make, model, round(avg(price)/avg(hp) ,2)as Avg_price
    from CarsT
    where make is not null and model  is not null
    group by make, model
    having Avg_price is not null
    order by Avg_price desc
    """
)

In [ ]:
Res9.show()

+-------------+---------------+---------+
|         make|          model|Avg_price|
+-------------+---------------+---------+
|      Estrima|           Birò|   2618.0|
|        Aixam|           City|   1990.0|
|       Ligier|          JS 50|   1914.0|
|      Ferrari|            F12|  1548.26|
|     Microcar|           M.Go|  1410.73|
|Mercedes-Benz|          S 650|  1138.22|
|      Maybach|        Pullman|  1049.03|
|     Microcar|             M8|  1024.92|
|     Microcar|            Due|   899.33|
|       Morgan|      3-Wheeler|   822.06|
|      Piaggio|            Ape|   765.83|
|        Aston|         Martin|   658.94|
|      Ferrari|            812|   550.56|
|       Zhidou|            D2S|   520.83|
|  Lamborghini|      Aventador|   507.52|
|      Renault|          Twizy|   504.09|
|      Bentley|    Flying Spur|   458.21|
|   Volkswagen|T6.1 California|   447.86|
|      McLaren|          675LT|   441.99|
|      Bentley|       Bentayga|   437.99|
+-------------+---------------+---

In [ ]:
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################

                                                              # SECTION 2 #

#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################

In this section i will work with txt files, but first i will try to cheat and create a schema for the file, in order to switch it to a dataframe and then work easily with it.

In [ ]:
file_tempreturerdd = sc.textFile("CorrectedTemperatures.txt")

In [ ]:
# file_tempreture = file_tempreturerdd.map(lambda row : row.split(","))

header = file_tempreturerdd.first()

file_tempreture3 = file_tempreturerdd.filter(lambda line : line != header)\
                                    .map(lambda row : row.split(","))\
                                    .map(lambda col: Row(
                                      date = col[0].strip(),
                                      city = col[1].strip(),
                                      country = col[2].strip(),
                                      max_temperature = float(col[3].strip()),
                                      min_temperature = float(col[4].strip())

                                        )
                                        )

# now we can create the data by simply
DF_TEMP = sqlsess.createDataFrame(file_tempreture3)


In this way we have our data as a pyspark dataframe file.

In [ ]:
DF_TEMP.show()

+----------+------+-------+---------------+---------------+
|      date|  city|country|max_temperature|min_temperature|
+----------+------+-------+---------------+---------------+
|2020/01/28| Paris| France|           23.0|           21.9|
|2020/01/13| Paris| France|           22.7|            9.6|
|2020/06/10| Paris| France|           29.5|           17.2|
|2020/06/27| Paris| France|            2.3|           -0.5|
|2020/05/11| Paris| France|           24.8|           17.8|
|2020/08/28|Berlin|Germany|           20.2|            6.2|
|2020/08/01|Berlin|Germany|           11.3|           -4.3|
|2020/04/06|Berlin|Germany|           31.0|           21.1|
|2020/11/09|Berlin|Germany|           28.9|           13.6|
|2020/08/28|Berlin|Germany|           23.7|           -5.6|
|2020/12/20|  Rome|  Italy|           29.3|           24.1|
|2020/10/17|  Rome|  Italy|           27.1|           19.6|
|2020/06/08|  Rome|  Italy|           24.7|           16.3|
|2020/01/23|  Rome|  Italy|           25

Now we will continue working on it as a RDD file to get the experience of working such files:

In [ ]:
# take the raw rdd file, filter the first row which contains the column's name and split it.

rdd_file = file_tempreturerdd.filter(lambda line: line  != header )\
                             .map(lambda row: row.split(","))

# the action in RDD is collect() to show the data as .show() in pyspark dataframe

rdd_file.collect()

[['2020/01/28', ' Paris', ' France', ' 23.0', ' 21.9'],
 ['2020/01/13', ' Paris', ' France', ' 22.7', ' 9.6'],
 ['2020/06/10', ' Paris', ' France', ' 29.5', ' 17.2'],
 ['2020/06/27', ' Paris', ' France', ' 2.3', ' -0.5'],
 ['2020/05/11', ' Paris', ' France', ' 24.8', ' 17.8'],
 ['2020/08/28', ' Berlin', ' Germany', ' 20.2', ' 6.2'],
 ['2020/08/01', ' Berlin', ' Germany', ' 11.3', ' -4.3'],
 ['2020/04/06', ' Berlin', ' Germany', ' 31.0', ' 21.1'],
 ['2020/11/09', ' Berlin', ' Germany', ' 28.9', ' 13.6'],
 ['2020/08/28', ' Berlin', ' Germany', ' 23.7', ' -5.6'],
 ['2020/12/20', ' Rome', ' Italy', ' 29.3', ' 24.1'],
 ['2020/10/17', ' Rome', ' Italy', ' 27.1', ' 19.6'],
 ['2020/06/08', ' Rome', ' Italy', ' 24.7', ' 16.3'],
 ['2020/01/23', ' Rome', ' Italy', ' 25.3', ' -7.6'],
 ['2020/12/03', ' Rome', ' Italy', ' 25.1', ' 22.2'],
 ['2020/05/16', ' Madrid', ' Spain', ' 26.0', ' 15.4'],
 ['2020/11/25', ' Madrid', ' Spain', ' 24.1', ' 17.3'],
 ['2020/06/28', ' Madrid', ' Spain', ' 27.5', ' 23.

indexing the data:

In [ ]:
date = rdd_file.map(lambda line: line[0])

date.collect()

['2020/01/28',
 '2020/01/13',
 '2020/06/10',
 '2020/06/27',
 '2020/05/11',
 '2020/08/28',
 '2020/08/01',
 '2020/04/06',
 '2020/11/09',
 '2020/08/28',
 '2020/12/20',
 '2020/10/17',
 '2020/06/08',
 '2020/01/23',
 '2020/12/03',
 '2020/05/16',
 '2020/11/25',
 '2020/06/28',
 '2020/09/04',
 '2020/12/04',
 '2020/10/12',
 '2020/06/02',
 '2020/06/02',
 '2020/12/28',
 '2020/03/11',
 '2020/09/20',
 '2020/02/29',
 '2020/02/17',
 '2020/03/05',
 '2020/02/15',
 '2020/06/08',
 '2020/07/11',
 '2020/04/05',
 '2020/10/14',
 '2020/06/24',
 '2020/05/14',
 '2020/10/06',
 '2020/07/01',
 '2020/09/18',
 '2020/10/10',
 '2020/05/03',
 '2020/04/28',
 '2020/08/16',
 '2020/09/13',
 '2020/01/22',
 '2020/01/13',
 '2020/12/19',
 '2020/07/20',
 '2020/09/28',
 '2020/07/16']

In [ ]:
date_greate_than_June = date.filter(lambda date: ( date > "2020/06/01") & (date < "2020/09/30") )
date_greate_than_June.collect()

['2020/06/10',
 '2020/06/27',
 '2020/08/28',
 '2020/08/01',
 '2020/08/28',
 '2020/06/08',
 '2020/06/28',
 '2020/09/04',
 '2020/06/02',
 '2020/06/02',
 '2020/09/20',
 '2020/06/08',
 '2020/07/11',
 '2020/06/24',
 '2020/07/01',
 '2020/09/18',
 '2020/08/16',
 '2020/09/13',
 '2020/07/20',
 '2020/09/28',
 '2020/07/16']

I want to see max tempretarue max, min temperature for the dates listet above

In [ ]:
def temp_min_max(array_temp):
  return [array_temp[0], array_temp[3], array_temp[4]]

data_greater = date_greate_than_June.collect()
summer_max_min = rdd_file.filter(lambda line: line[0] in data_greater)\
                          .map(lambda row: temp_min_max(row))

remember that we have made this step
data_greater = date_greate_than_June.collect()
befor the filtering since we cant include an rdd operation in another rdd one

In [ ]:
summer_max_min.collect()

[['2020/06/10', ' 29.5', ' 17.2'],
 ['2020/06/27', ' 2.3', ' -0.5'],
 ['2020/08/28', ' 20.2', ' 6.2'],
 ['2020/08/01', ' 11.3', ' -4.3'],
 ['2020/08/28', ' 23.7', ' -5.6'],
 ['2020/06/08', ' 24.7', ' 16.3'],
 ['2020/06/28', ' 27.5', ' 23.0'],
 ['2020/09/04', ' 34.0', ' 17.0'],
 ['2020/06/02', ' 23.0', ' 16.7'],
 ['2020/06/02', ' 9.6', ' 8.6'],
 ['2020/09/20', ' 26.6', ' 20.9'],
 ['2020/06/08', ' 30.5', ' -8.2'],
 ['2020/07/11', ' 29.7', ' 6.2'],
 ['2020/06/24', ' 24.3', ' 21.4'],
 ['2020/07/01', ' -2.2', ' -7.4'],
 ['2020/09/18', ' 21.8', ' -2.5'],
 ['2020/08/16', ' 22.0', ' 21.6'],
 ['2020/09/13', ' 27.1', ' -5.9'],
 ['2020/07/20', ' 31.0', ' -6.4'],
 ['2020/09/28', ' 9.4', ' -3.9'],
 ['2020/07/16', ' 23.5', ' 9.2']]

Write the scheme of an Hadoop MapReduce applicative program to address the following question.
Considering all historical data stored in Temperatures.txt, the application must select the cities with at least
one date with maximum temperature greater than 35°C and at least one date with minimum temperature less
than -5°C.
The two observations may be the same date or different dates.
Store the results in an HDFS folder.
The output should contain one line for each of the selected cities.

In [ ]:
rdd_file.collect()

[['2020/01/28', ' Paris', ' France', ' 23.0', ' 21.9'],
 ['2020/01/13', ' Paris', ' France', ' 22.7', ' 9.6'],
 ['2020/06/10', ' Paris', ' France', ' 29.5', ' 17.2'],
 ['2020/06/27', ' Paris', ' France', ' 2.3', ' -0.5'],
 ['2020/05/11', ' Paris', ' France', ' 24.8', ' 17.8'],
 ['2020/08/28', ' Berlin', ' Germany', ' 20.2', ' 6.2'],
 ['2020/08/01', ' Berlin', ' Germany', ' 11.3', ' -4.3'],
 ['2020/04/06', ' Berlin', ' Germany', ' 31.0', ' 21.1'],
 ['2020/11/09', ' Berlin', ' Germany', ' 28.9', ' 13.6'],
 ['2020/08/28', ' Berlin', ' Germany', ' 23.7', ' -5.6'],
 ['2020/12/20', ' Rome', ' Italy', ' 29.3', ' 24.1'],
 ['2020/10/17', ' Rome', ' Italy', ' 27.1', ' 19.6'],
 ['2020/06/08', ' Rome', ' Italy', ' 24.7', ' 16.3'],
 ['2020/01/23', ' Rome', ' Italy', ' 25.3', ' -7.6'],
 ['2020/12/03', ' Rome', ' Italy', ' 25.1', ' 22.2'],
 ['2020/05/16', ' Madrid', ' Spain', ' 26.0', ' 15.4'],
 ['2020/11/25', ' Madrid', ' Spain', ' 24.1', ' 17.3'],
 ['2020/06/28', ' Madrid', ' Spain', ' 27.5', ' 23.

In [ ]:
## the map should take each line, then it should return the (city, char), so we can use the map funtion to generate the data for all cities that have passsed one of
## the two conditions -- > 35 and -- < 5

## then in the reduce we only return these with both conditions satisfied.

def map(line):

  field = line.split(",")
  city = line[1]
  max_temp = line[3]
  min_temp = line[4]

  if (int(max_temp) > 35):
    yield(city, "g")

  elif (int(min_temp) < -5 ):
    yield (city, "l")


#reduce function where we need to check wheter the values of city contain both "l" and "g"

def reduce(key, values):

    greater = False
    less =  False

    for val in values:
      if val == "g":
        greater = True

      elif val == "l":
        less = True

    if (greater & less): yield key






In [ ]:

#finding the max average
only_max = summer_max_min.map(lambda row: row[1])
total_max = only_max.reduce(lambda x,y: float(x)+float(y))
average_temp_max = round(total_max / only_max.count(), 2)

#finding the min average
only_min = summer_max_min.map(lambda row: row[2])
total_min = only_min.reduce(lambda x,y: float(x)+float(y))
average_temp_min = round(total_min / only_min.count(), 2)

#printing the averages
print(f"Average temperature for max {average_temp_max}, average temperature for min {average_temp_min}")

Average temperature for max 21.4, average temperature for min 6.65


The application must first select the historical temperature values observed during the summer of 2019.
Based on the resulting dataset, the application must store in HDFS the average value of max_temperature
for each city. The format of the output must be “(city-country, average max_temperature of city)” where citycountry is the concatenation of the city and its country.

In [ ]:

summer_data = DF_TEMP.filter( (f.col("date") > "2020/06/01") & (f.col("date") < "2020/09/30"))

result = summer_data.withColumn("city_country", f.concat_ws("-", f.col("city"), f.col("country"))) \
                    .groupBy("city_country") \
                    .agg(f.avg("max_temperature").alias("avg_max_temperature"))

In [ ]:
result.show()

+--------------------+-------------------+
|        city_country|avg_max_temperature|
+--------------------+-------------------+
|        Madrid-Spain|              30.75|
|      Berlin-Germany| 18.400000000000002|
|        Paris-France|               15.9|
|Prague-Czech Repu...|               26.6|
|          Rome-Italy|               24.7|
|      Vienna-Austria|               16.3|
|       Athens-Greece|              24.55|
|       Warsaw-Poland| 28.166666666666668|
|    Budapest-Hungary|                9.8|
|     Lisbon-Portugal|               21.3|
+--------------------+-------------------+



find cities where the difference between average max value in the summer and winter is more than 20 C

In [ ]:
temp_summer = DF_TEMP.filter( (f.col("date") > "2020/06/01") & (f.col("date") < "2020/08/30"))

summer_dates = temp_summer.select("date").distinct()

temp_winter = DF_TEMP.join(summer_dates, ["date"], "left_anti")

# prpare summer data
summer_data = temp_summer.groupBy("city").agg(f.round(f.avg(f.col("max_temperature")),2).alias("summer_max"))

winter_data = temp_winter.groupBy("city").agg(f.round(f.avg(f.col("max_temperature")),2).alias("winter_max"))

grouped = summer_data.join(winter_data, ["city"])

cities = grouped.select("city").filter(f.abs(f.col("summer_max") - f.col("winter_max")) > 20)

cities.show()

+--------+
|    city|
+--------+
|Budapest|
+--------+

